#Solver - Problema 1

In [ ]:
!pip install -q pyomo
!wget -N -q "https://minlp.com/downloads/xecs/baron/current/baron-lin64.zip"
!unzip baron-lin64.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.6 MB/s eta 0:00:00
Archive:  baron-lin64.zip
   creating: baron-lin64/
  inflating: baron-lin64/README.txt  
  inflating: baron-lin64/BARON EULA.pdf  
  inflating: baron-lin64/baron       
  inflating: baron-lin64/baron manual.pdf  
   creating: baron-lin64/examples/
  inflating: baron-lin64/examples/nlp3.bar  
  inflating: baron-lin64/examples/robot.bar  
  inflating: baron-lin64/examples/gear.bar  
  inflating: baron-lin64/examples/scqp1.bar  
  inflating: baron-lin64/examples/pool1.bar  
  inflating: baron-lin64/examples/nlp1.bar  
  inflating: baron-lin64/examples/milp.bar  
  inflating: baron-lin64/examples/minlp.bar  
  inflating: baron-lin64/examples/gear20.bar  
  inflating: baron-lin64/examples/nlp2.bar  
  inflating: baron-lin64/examples/iqp1.bar  


$\min x^2+2y^2-4x-4y+xy+120$

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=Reals)
model.y = Var(domain=Reals)

# declare objective
model.obj = Objective(expr = model.x**2 + 2*model.y**2 -4*model.x -4*model.y +model.x*model.y +120, sense=minimize)

model.pprint()

msolver = SolverFactory('baron', executable="/content/baron-lin64/baron")
msolver.solve(model, tee=True)

# display solution
print('\nUtilidade = ', model.obj())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x**2 + 2*y**2 - 4*x - 4*y + x*y + 120

3 Declarations: x y obj
 BARON version 23.6.23. Built: LNX-64 Fri Jun 23 12:56:02 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 No BARON license file found in user PATH.  Continuing in demo mode.
 Model size is allowable within BARON demo size.

 If you use this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, A. and N. V. Sahinidis,
 A hybrid LP/NLP paradigm for

#Método do Gradiente

$f(x,y) = x^2+2y^2-4x-4y+xy+120$

$\nabla f(x,y) = \begin{bmatrix}
2x + y -4 \\
x + 4y -4
\end{bmatrix}$

In [ ]:
import numpy as np

def grad(xn):
  x = xn.item(0)
  y = xn.item(1)
  dx = 2*x+y-4
  dy = x+4*y-4
  return np.matrix([[dx],[dy]])

xn = np.matrix([[2],[2]])

w = 0.1
maxIter = 100

for n in range(maxIter):
  xn = xn - w*grad(xn)

print(xn)

[[1.71428571]
 [0.57142857]]


## Usando pytorch

In [ ]:
# import torch library
import torch
import numpy as np

def grad(xn):
  x1 = xn[0]
  y1 = xn[1]
  x = torch.tensor(x1, requires_grad = True)
  y = torch.tensor(y1, requires_grad = True)
  f = x**2 + 2*y**2 - 4*x -4*y +x*y + 120
  f.backward()
  dx = x.grad
  dy = y.grad
  return np.matrix([[dx.item()],[dy.item()]])

xn = np.matrix([[2.0],[2.0]])

w = 0.1
maxIter = 100

for n in range(maxIter):
  xn = xn - w*grad(xn)

print(xn)

[[1.71212706]
 [0.57232475]]


#Método de Newton

$f(x,y) = x^2+2y^2-4x-4y+xy+120$

$\nabla f(x,y) = \begin{bmatrix}
2x + y -4 \\
x + 4y -4
\end{bmatrix}$

$H = \begin{bmatrix}
2 & 1\\
1 & 4
\end{bmatrix}$

In [ ]:
import numpy as np

def grad(xn):
  x = xn.item(0)
  y = xn.item(1)
  a1 = 2*x+y-4
  a2 = x+4*y-4
  return np.matrix([[a1],[a2]])

H = np.matrix([[2,1],[1,4]])
H_1 = np.linalg.inv(H)

xn = np.matrix([[2],[2]])

w = 0.1
maxIter = 100

for n in range(maxIter):
  xn = xn - w*H_1*grad(xn)
  #print(grad(xn))

print(xn)

[[1.8]
 [5.4]]
[[1.62]
 [4.86]]
[[1.458]
 [4.374]]
[[1.3122]
 [3.9366]]
[[1.18098]
 [3.54294]]
[[1.062882]
 [3.188646]]
[[0.9565938]
 [2.8697814]]
[[0.86093442]
 [2.58280326]]
[[0.77484098]
 [2.32452293]]
[[0.69735688]
 [2.09207064]]
[[0.62762119]
 [1.88286358]]
[[0.56485907]
 [1.69457722]]
[[0.50837317]
 [1.5251195 ]]
[[0.45753585]
 [1.37260755]]
[[0.41178226]
 [1.23534679]]
[[0.37060404]
 [1.11181211]]
[[0.33354363]
 [1.0006309 ]]
[[0.30018927]
 [0.90056781]]
[[0.27017034]
 [0.81051103]]
[[0.24315331]
 [0.72945993]]
[[0.21883798]
 [0.65651393]]
[[0.19695418]
 [0.59086254]]
[[0.17725876]
 [0.53177629]]
[[0.15953289]
 [0.47859866]]
[[0.1435796 ]
 [0.43073879]]
[[0.12922164]
 [0.38766491]]
[[0.11629947]
 [0.34889842]]
[[0.10466953]
 [0.31400858]]
[[0.09420257]
 [0.28260772]]
[[0.08478232]
 [0.25434695]]
[[0.07630408]
 [0.22891225]]
[[0.06867368]
 [0.20602103]]
[[0.06180631]
 [0.18541893]]
[[0.05562568]
 [0.16687703]]
[[0.05006311]
 [0.15018933]]
[[0.0450568]
 [0.1351704]]
[[0.04055112]


# Problema 2

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x = Var(domain=Integers)
model.y = Var(domain=Integers)

# declare objective
model.obj = Objective(expr = -model.x**2 - model.y**2 +model.x*model.y +6*model.x +2*model.y, sense=maximize)

model.restr1 = Constraint(expr = 2*model.x + 2*model.y == 12)

model.pprint()

msolver = SolverFactory('baron', executable="/content/baron-lin64/baron")
msolver.solve(model, tee=True)

# display solution
print('\nUtilidade = ', model.obj())

print('\nDecision Variables')
print('x = ', model.x())
print('y = ', model.y())

2 Var Declarations
    x : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True : Integers
    y : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True : Integers

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : - x**2 - y**2 + x*y + 6*x + 2*y

1 Constraint Declarations
    restr1 : Size=1, Index=None, Active=True
        Key  : Lower : Body      : Upper : Active
        None :  12.0 : 2*x + 2*y :  12.0 :   True

4 Declarations: x y obj restr1
 BARON version 23.6.23. Built: LNX-64 Fri Jun 23 12:56:02 EDT 2023

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 No BARON license file found in user PATH.  Continuing in demo mode.
 Model size is allowable within BARON demo siz

#KKT

##Problema 3

$\max z = -24x_1^2 -24x_2^2 +240x_1 + 240x_2\\
s.t.\\
2x_1+3x_2 \leq 18 \\
x_1 \leq 6 \\
x_1,x_2 \geq 0$

$\min 24x_1^2 +24x_2^2 -240x_1 - 240x_2\\
s.t.\\
2x_1+3x_2 -18 \leq 0 \\
x_1 -6 \leq 0 \\
- x_1 \leq 0 \\
- x_2 \leq 0$

$\mathcal{L}(x_1,x_2,\mu_1,\mu_2) = 24x_1^2+24x_2^2-240x_1-240x_2 + \mu_1(2x_1+3x_2-18) + \mu_2(x_1-6) + \mu_3(-x_1) + \mu_4(-x_2)$

**Estacionariedade**

$48x^*_1-240+2\mu^*_1 + \mu^*_2 -\mu^*_3= 0\\
48x^*_2-240+3\mu^*_1-\mu^*_4 = 0$

**Viabilidade Primal**

$2x_1+3x_2 -18 \leq 0 \\
x_1 -6 \leq 0 \\
-x_1 \leq 0 \Rightarrow x^*_1 \geq 0 \\
-x_2 \leq 0 \Rightarrow x^*_2 \geq 0$

**Viabilidade Dual**

$\mu^*_1,\mu^*_2,\mu^*_3,\mu^*_4 \geq 0$

**Complementaridade**

$\mu^*_1(2x_1+3x_2 -18) = 0\\
\mu^*_2(x_1 -6) = 0\\
\mu^*_3(-x^*_1) = 0\\
\mu^*_4(-x^*_2) = 0$

$
\mu^*_3(-x^*_1) = 0 \Rightarrow \mu^*_3(x^*_1) = 0 \Rightarrow x^*_1(48x^*_1-240+2\mu^*_1 + \mu^*_2) = 0 \\
\mu^*_4(-x^*_2) = 0 \Rightarrow \mu^*_4(x^*_2) = 0 \Rightarrow x^*_2(48x^*_2-240+3\mu^*_1) = 0
$

---
Caso $x^*_1 \neq 0$, $x^*_2 \neq 0$, $\mu^*_1 \neq 0$, $\mu^*_2 \neq 0$

\\
$
48x^*_1-240+2\mu^*_1 + \mu^*_2 = 0 \\
48x^*_2-240+3\mu^*_1 = 0 \\
2x_1+3x_2 -18 = 0 \\
x_1 -6 = 0
$

\\
$
x_1 = 6 \\
x_2 = 2 \\
\mu^*_1 = 48 \\
\mu^*_2 = -144
$

---
Caso $x^*_1 = 0$, $x^*_2 \neq 0$, $\mu^*_1 \neq 0$, $\mu^*_2 \neq 0$

\\
$
48x^*_2-240+3\mu^*_1 = 0 \\
3x_2 -18 = 0
$

\\
$
x_2 = 6 \\
\mu^*_1 = -16
$

---
Caso $x^*_1 \neq 0$, $x^*_2 = 0$, $\mu^*_1 \neq 0$, $\mu^*_2 \neq 0$

\\
$
48x^*_1-240+2\mu^*_1 + \mu^*_2 = 0 \\
2x_1 -18 = 0 \\
x_1 -6 = 0
$

\\
Se $\mu^*_1 = 0$ então $x^*_1 = 6$ e $\mu^*_2 = -48$ que é **FALSO**

Se $\mu^*_2 = 0$ então $x^*_1 = 9$ e $\mu^*_1 = -96$ que é **FALSO**

---
Caso $x^*_1 \neq 0$, $x^*_2 \neq 0$, $\mu^*_1 = 0$, $\mu^*_2 \neq 0$

\\
$
48x^*_1-240 + \mu^*_2 = 0 \\
48x^*_2-240 = 0 \\
x_1 -6 = 0
$

\\
$
x_1 = 6 \\
x_2 = 5 \\
\mu^*_2 = -48
$

---
Caso $x^*_1 \neq 0$, $x^*_2 \neq 0$, $\mu^*_1 \neq 0$, $\mu^*_2 = 0$

\\
$
48x^*_1-240+2\mu^*_1 = 0 \\
48x^*_2-240+3\mu^*_1 = 0 \\
2x_1+3x_2 -18 = 0
$

\\
$
x_1 = 3.92 \\
x_2 = 3.38 \\
\mu^*_1 = 25.85 \\
\mu^*_2 = 0
$


#Solver - Problema 3

$\max z = -24x_1^2 -24x_2^2 +240x_1 + 240x_2\\
s.t.\\
2x_1+3x_2 \leq 18 \\
x_1 \leq 6 \\
x_1,x_2 \geq 0$

In [ ]:
!pip install -q cplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 25.9 MB/s eta 0:00:00


In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x1 = Var()
model.x2 = Var()

# declare objective
model.obj = Objective(expr = -24*model.x1**2-24*model.x2**2+240*model.x1+240*model.x2, sense=maximize)

model.c1 = Constraint(expr = 2*model.x1 + 3*model.x2 <= 18)
model.c2 = Constraint(expr = model.x1 <= 6)
model.c3 = Constraint(expr = model.x1 >= 0)
model.c4 = Constraint(expr = model.x2 >= 0)

model.pprint()

#msolver = SolverFactory('cplex_direct')
msolver = SolverFactory('baron', executable="/content/baron-lin64/baron")
msolver.solve(model, tee=True)

# display solution
print('\nUtilidade = ', model.obj())

print('\nDecision Variables')
print('x1 = ', model.x1())
print('x2 = ', model.x2())


2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :  None :  None :  None : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : -24*x1**2 - 24*x2**2 + 240*x1 + 240*x2

4 Constraint Declarations
    c1 : Size=1, Index=None, Active=True
        Key  : Lower : Body        : Upper : Active
        None :  -Inf : 2*x1 + 3*x2 :  18.0 :   True
    c2 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :  -Inf :   x1 :   6.0 :   True
    c3 : Size=1, Index=None, Active=True
        Key  : Lower : Body : Upper : Active
        None :   0.0 :   x1 :  +Inf :   True
    c4 : Size=1, Index=None, Active=True
        Key  :

# Problema 4

$\max x_1^2 + x_2^2 - x_1x_2 +2x_1 +4x_2 + 3\\
s.t.\\
-5 \leq x_1 \leq 5\\
-5 \leq x_2 \leq 5\\
$

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# declare decision variables
model.x1 = Var(domain=Reals,bounds=(-5,5))
model.x2 = Var(domain=Reals,bounds=(-5,5))

# declare objective
model.obj = Objective(expr = model.x1**2 + model.x2**2 -model.x1*model.x2 +2*model.x1 + 4*model.x2 +3, sense=maximize)

model.pprint()

msolver = SolverFactory('baron', executable="/content/baron-lin64/baron")
msolver.solve(model, tee=True)

# display solution
print('\nUtilidade = ', model.obj())

print('\nDecision Variables')
print('x = ', model.x1())
print('y = ', model.x2())

2 Var Declarations
    x1 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    -5 :  None :     5 : False :  True :  Reals
    x2 : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :    -5 :  None :     5 : False :  True :  Reals

1 Objective Declarations
    obj : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : maximize : x1**2 + x2**2 - x1*x2 + 2*x1 + 4*x2 + 3

3 Declarations: x1 x2 obj
 BARON version 24.4.6. Built: LNX-64 Sat Apr 6 10:35:39 EDT 2024 
 Running on machine f1463a1f0fcd

 BARON is a product of The Optimization Firm.
 For information on BARON, see https://minlp.com/about-baron
 No BARON license file found in user PATH.  Continuing in demo mode.
 Model size is allowable within BARON demo size.

 If you publish work using this software, please cite publications from
 https://minlp.com/baron-publications, such as: 

 Khajavirad, 